# Workbook 3 - Extracting UrbanSound8K audio features for a Convolutional Neural Network

First, import the libraries. As with the earlier notebooks, the audio processing is handled by a library called librosa, if you haven't already installed it on your local system, do that with: pip install librosa

> 먼저 라이브러리를 가져옵니다. 이전 노트북과 마찬가지로 오디오 처리는 librosa라는 라이브러리에서 처리합니다. 로컬 시스템에 아직 설치하지 않은 경우에는 다음을 수행하십시오. pip install librosa

You'll also need Keras and Tensorflow installed. This was updated on 23 July 2017 to use Python3, Keras 2 and Tensorflow 1.2.

In [1]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

Notebook 1 explained how to extract and aggregate several audio features using processing methods provided by the librosa library which could be fed into a Feed-Forward Network.

> 노트북 1은 피드 포워드 네트워크에 공급 될 수있는 librosa 라이브러리가 제공하는 처리 방법을 사용하여 여러 오디오 기능을 추출하고 집계하는 방법을 설명했습니다.

This notebook is going to describe how to learn audio data using a different architecture, a Convolutional Neural Network (CNN).

> 이 노트북에서는 다른 아키텍처 인 CNN (Convolutional Neural Network)을 사용하여 오디오 데이터를 배우는 방법을 설명합니다.

A CNN organises hidden units to take advantage of the local structure present in two-dimensional input data (the classic example being edges in images). By concentrating on identifying local features each hidden unit only needs to process a tiny part of the whole input space, instead of being connected to all the inputs coming from the previous layer. Processing proceeds by successively considering small windows of the data set, (e.g. 3x3 pixels), called the receptive field.

> CNN은 2 차원 입력 데이터에있는 로컬 구조를 활용하기 위해 숨겨진 단위를 구성합니다 (이미지의 가장자리 인 전형적인 예). 로컬 피처를 식별하는 데 집중함으로써 각각의 숨겨진 유닛은 이전 레이어에서 오는 모든 입력에 연결되는 대신 전체 입력 공간의 작은 부분만 처리하면 됩니다. 처리는 수용 필드라 불리는 데이터 세트의 작은 윈도우 (예를 들어, 3x3 픽셀)를 연속적으로 고려함으로써 진행된다.

The weights of hidden units create a convolutional kernel (or filter) which is applied to the whole input space, like a succession of tiles, resulting in a feature map. As a result, one set of weights can be reused for the whole input space. As a feature like an edge can occur anywhere in the input space, the CNN approach greatly reduces the number of parameters required, as well as improving the model's robustness.

> 숨겨진 유닛의 가중치는 연속 된 타일처럼 전체 입력 공간에 적용되는 컨볼 루션 커널 (또는 필터)을 작성하여 피쳐 맵을 만듭니다. 결과적으로 한 세트의 가중치가 전체 입력 공간에 재사용 될 수 있습니다. 가장자리와 같은 기능이 입력 공간의 어디에서나 발생할 수 있으므로 CNN 접근 방식은 필요한 매개 변수 수를 크게 줄이고 모델의 견고성을 향상시킵니다.

A typical convolutional layer will consist of numerous filters (feature maps). Further dimensionality reduction can be achieved through pooling layers, which merge adjacent cells of a feature map, using pooling operations such as max (winner takes all) or the mean of the input cells. This downsampling further improves the tolerance of the network to variation and noise.

> 전형적인 길쌈 계층은 수많은 필터 (feature map)로 구성됩니다. 차원 구조 축소는 최대 (승자가 차지함) 또는 입력 셀의 평균과 같은 풀링 연산을 사용하여 피쳐 맵의 인접 셀을 병합하는 레이어 풀링을 통해 달성 할 수 있습니다. 이 다운 샘플링은 변동 및 잡음에 대한 네트워크의 허용 오차를 더욱 개선합니다.

CNNs have proved especially successful at classification tasks, particularly of images, but to use them to classify audio files, we'll need some way of extracting audio features in such a way that they reflect the kind of input data a CNN expects. The method is explained by Karol J. Piczak in his paper Environmental sound classification with convolutional neural networks (http://karol.piczak.com/papers/Piczak2015-ESC-ConvNet.pdf). This describes how to get equal size segments from varying length audio clips, and which audio features can be fed into the network as separate channels (akin to RGB channels of colour images).

> CNN은 특히 이미지의 분류 작업에서 특히 성공적 이었지만 오디오 파일을 분류하는 데 사용하려면 CNN에서 기대하는 입력 데이터의 종류를 반영하는 방식으로 오디오 기능을 추출해야합니다. 이 방법은 Karol J. Piczak이 그의 논문에서 Convolutional Neural Network (http://karol.piczak.com/papers/Piczak2015-ESC-ConvNet.pdf)를 이용한 환경 사운드 분류에 설명되어 있습니다. 이것은 다양한 길이의 오디오 클립에서 동일한 크기의 세그먼트를 얻는 방법과 오디오 기능을 네트워크에 별도의 채널로 제공 할 수있는 방법을 설명합니다 (RGB 이미지 채널 색상과 유사).